# Create and run a ZONEBUDGET on a general MODFLOW model

This notebook implements a grid-search approach to finding hydraulic conductivities that result in heads that seem reasonable.

Project specific variables are imported in the model_spec.py and gen_mod_dict.py files that must be included in the notebook directory. The first first includes pathnames to data sources that will be different for each user. The second file includes a dictionary of model-specific information such as  cell size, default hydraulic parameter values, and scenario defintion (e.g. include bedrock, number of layers, etc.). There are examples in the repository. Run the following cells up to the "Run to here" cell to get a pull-down menu of models in the model_dict. Then, without re-running that cell, run all the remaining cells.  Re-running the following cell would re-set the model to the first one in the list, which you probably don't want. If you use the notebook option to run all cells below, it runs the cell you're in, so if you use that option, move to the next cell (below the pull-down menu of models) first.

In [1]:
__author__ = 'Jeff Starn'
%matplotlib notebook
from model_specs import *
from gen_mod_dict import *

import os, sys
import shutil
import numpy as np
import matplotlib.pyplot as plt

from flopy.utils.postprocessing import get_water_table


import flopy.utils.binaryfile as bf
from matplotlib import colors
import flopy as fp
import pandas as pd
# import ipyparallel as ipp
# from model_specs import *
# from gen_mod_dict import *

from ipywidgets import interact, Dropdown
from IPython.display import display

The next cell is a template for making this notebook into a batch script. To do so, save this notebook as a .py file and edit it as follows. Comment out all the notebook-specific commands (drop-down menu stuff and commands preceded by %). Indent everything below the next cell twice so that it falls within the 'for' loop and the 'try' statement. Move the 'except' statement to the end of the script. Comment out lines in the cell after 'Preliminary stuff' so that the model is selected in the 'for' loop from gen_mod_dict. You can leave the print statement in that cell uncommented. 

In [2]:
for key, value in model_dict.items():   # from "gen_mod_dict.py"
    md = key
    ms = model_dict[md]
    print('trying {}'.format(md))
    try:
        pass
    except:
        pass

trying Assabet
trying CoastalCT


In [3]:
models = list(model_dict.keys())
models.sort()
model_area = Dropdown(
    options=models,
    description='Model:',
    background_color='cyan',
    border_color='black',
    border_width=2)
display(model_area)

### Run to here to initiate notebook

First time using this notebook in this session (before restarting the notebook), run the cells up to this point. Then select your model from the dropdown list above. Move your cursor to this cell and use the toolbar menu Cell --> Run All Below.  After the first time, if you want to run another model, select your model and start running from this cell--you don't need to re-run the cells from the beginning.

## Preliminary stuff

In [58]:
md = model_area.value
ms = model_dict[md]
print('The model being processed is {}\n'.format(md))

The model being processed is CoastalCT



In [59]:
hydro_wt = 1.0
calib=False
PEST=True
label = "tillBedrock_0_round_params_RIV"
label="lowGHBConductance"
label=""
# label="lowGHBConductance_lowGHBConductance"

geo_ws = os.path.join(proj_dir, ms['ws'])
parent_ws = os.path.join(geo_ws, scenario_dir)
if calib:
    dir_name = '{}_cal_wt_{:4.2f}'.format(scenario_dir, hydro_wt)
else:
    dir_name = scenario_dir
    
if PEST:
    dir_name = dir_name + "_PEST"
    
if label!='':
    dir_name = dir_name +"_"+str(label)
    
model_ws = os.path.join(geo_ws, dir_name)

if not os.path.exists(model_ws):
    print('This calibration scenario has not been completed')

In [60]:
model_file = os.path.join(geo_ws, 'model_grid.csv')
model_grid = pd.read_csv(model_file, index_col='node_num', na_values=['nan', hnoflo])

NROW = model_grid.row.max() + 1
NCOL = model_grid.col.max() + 1
num_cells = NROW * NCOL
NLAY = num_surf_layers + 1

zones = model_grid.HUC12_shortCD_ext.values.reshape(NROW, NCOL)
zones = zones.astype(int)
zone_array = np.repeat(zones[np.newaxis,:,:],NLAY,axis=0)

#change 0 to 1000
zone_array[zone_array==0]=1000


In [61]:
cbc_file = os.path.join(model_ws,'{}.cbc'.format(md))
zb = fp.utils.zonbud.ZoneBudget(cbc_file,zone_array)

C:\Users\jbarclay\.conda\envs\genmod\lib\site-packages\flopy\utils\zonbud.py:1424: RuntimeWarning: invalid value encountered in double_scalars
  flux = 100 * in_minus_out / (in_plus_out / 2.)


In [62]:
#extract the zonebudget results and change it to 1 row per zone
names = zb.get_record_names()
names = names.tolist()
nameLst = [x for x in names if "ZONE" not in x]
zb_df = zb.get_dataframes(names=nameLst)
zb_df.reset_index(level="totim", drop=True, inplace=True)
zb_df = zb_df.transpose()
zb_df['Zone_Nm']=zb_df.index
zb_df.reset_index(drop=True,inplace=True)

In [63]:
#add a zone column and populate it with the zone number
zb_df['Zone']=[int(x) for x in (zb_df.Zone_Nm.str[5:])]
#change zone 0 back to 0
#zb_df['Zone']==1000
zb_df.loc[zb_df.Zone==1000,'Zone']=0

#calculate some new columns
zb_df['TRANSFERS_IN']=zb_df['TOTAL_IN']-(zb_df[['CONSTANT_HEAD_IN', 'DRAINS_IN','HEAD_DEP_BOUNDS_IN', 'RECHARGE_IN',]].sum(axis=1))
zb_df['TRANSFERS_OUT']=zb_df['TOTAL_OUT']-(zb_df[['CONSTANT_HEAD_OUT', 'DRAINS_OUT','HEAD_DEP_BOUNDS_OUT', 'RECHARGE_OUT',]].sum(axis=1))
zb_df['CGWD_FRAC']=zb_df['HEAD_DEP_BOUNDS_OUT']/zb_df['TOTAL_OUT']
zb_df['DRN_FRAC']=zb_df['DRAINS_OUT']/zb_df['TOTAL_OUT']
zb_df['RIV_IN_FRAC']= zb_df['RIVER_LEAKAGE_IN']/zb_df['TOTAL_IN']
zb_df['TRANS_OUT_FRAC']= zb_df['TRANSFERS_OUT']/zb_df['TOTAL_OUT']
zb_df['TRANS_IN_FRAC']= zb_df['TRANSFERS_IN']/zb_df['TOTAL_IN']
zb_df['CGWD_DRN_ratio']=zb_df['HEAD_DEP_BOUNDS_OUT']/zb_df['DRAINS_OUT']

#change inf values to nan
zb_df.loc[np.isinf(zb_df.CGWD_DRN_ratio),'CGWD_DRN_ratio']=np.nan

In [10]:
f_out = os.path.join(model_ws, 'Zone_Output.csv')
zb_df.to_csv(f_out)

In [65]:
zb_df['CGWD_In_Per']=zb_df.HEAD_DEP_BOUNDS_IN/zb_df.TOTAL_IN*100.0

In [66]:
zb_df.sort_values(by=['CGWD_In_Per'], ascending=False)

name,CONSTANT_HEAD_IN,CONSTANT_HEAD_OUT,DRAINS_IN,DRAINS_OUT,HEAD_DEP_BOUNDS_IN,HEAD_DEP_BOUNDS_OUT,RECHARGE_IN,RECHARGE_OUT,RIVER_LEAKAGE_IN,RIVER_LEAKAGE_OUT,...,Zone,TRANSFERS_IN,TRANSFERS_OUT,CGWD_FRAC,DRN_FRAC,TRANS_OUT_FRAC,TRANS_IN_FRAC,CGWD_DRN_ratio,CGWD_In_Per,RIV_IN_FRAC
100,0.0,0.0,0.000000,63617.945312,7092.415039,142691.078125,197397.671875,0.0,232416.656250,232416.656250,...,852.450378,ZONE_101,101,27926.562500,26107.625000,0.613945,0.273724,0.112331,2.242938,3.051595
81,0.0,0.0,0.000000,38284.273438,1265.488647,18048.048828,55593.394531,0.0,58828.738281,58828.734375,...,81.322128,ZONE_82,82,1969.855469,2496.414062,0.306790,0.650775,0.042435,0.471422,2.151140
20,0.0,0.0,0.000000,60866.652344,1747.764404,19038.884766,84818.250000,0.0,92150.140625,92150.171875,...,4776.916504,ZONE_21,21,5584.125000,12244.632812,0.206607,0.660516,0.132877,0.312797,1.896649
105,0.0,0.0,0.000000,11570.684570,1300.638306,58073.324219,52544.261719,0.0,74767.718750,74795.265625,...,114.413948,ZONE_106,106,20922.820312,5151.257812,0.776430,0.154698,0.068871,5.019005,1.739572
29,0.0,0.0,0.000000,91861.421875,2518.452637,92737.687500,195470.468750,0.0,206034.593750,206034.578125,...,1295.222656,ZONE_30,30,8045.671875,21435.468750,0.450107,0.445854,0.104038,1.009539,1.222345
4,0.0,0.0,0.000000,4006.442871,172.358017,14620.751953,20464.656250,0.0,21499.570312,21499.570312,...,8.163896,ZONE_5,5,862.556641,2872.375000,0.680049,0.186350,0.133602,3.649310,0.801681
41,0.0,0.0,0.000000,52306.789062,921.689209,57996.855469,108074.859375,0.0,119216.476562,119216.484375,...,286.127625,ZONE_42,42,10219.929688,8912.843750,0.486484,0.438755,0.074762,1.108783,0.773122
76,0.0,0.0,0.000000,65237.406250,496.668060,37103.539062,105813.203125,0.0,110902.851562,110906.203125,...,4190.284668,ZONE_77,77,4592.976562,8565.257812,0.334549,0.588221,0.077230,0.568746,0.447841
72,0.0,0.0,0.000000,51512.078125,469.831512,68802.593750,124528.304688,0.0,131015.648438,131015.671875,...,1022.027161,ZONE_73,73,6017.515625,10701.000000,0.525148,0.393175,0.081677,1.335660,0.358607
11,0.0,0.0,0.000000,98000.906250,465.535583,31234.904297,138800.343750,0.0,143634.437500,143634.453125,...,2202.728760,ZONE_12,12,4368.562500,14398.640625,0.217461,0.682294,0.100245,0.318721,0.324111


In [67]:
zb_df[zb_df.Zone>70].sort_values(by=['Zone'], ascending=False)

name,CONSTANT_HEAD_IN,CONSTANT_HEAD_OUT,DRAINS_IN,DRAINS_OUT,HEAD_DEP_BOUNDS_IN,HEAD_DEP_BOUNDS_OUT,RECHARGE_IN,RECHARGE_OUT,TOTAL_IN,TOTAL_OUT,...,WELLS_OUT,Zone_Nm,Zone,TRANSFERS_IN,TRANSFERS_OUT,CGWD_FRAC,DRN_FRAC,TRANS_FRAC,CGWD_DRN_ratio,CGWD_In_Per
108,0.0,0.0,0.000000,80788.640625,10.239424,52350.882812,128162.671875,0.0,142332.562500,142332.578125,...,3693.382080,ZONE_109,109,14159.648438,9193.046875,0.367807,0.567605,0.064588,0.647998,0.007194
107,0.0,0.0,0.000000,43529.757812,130.972382,26492.619141,68946.617188,0.0,76299.484375,76316.421875,...,386.095337,ZONE_108,108,7221.898438,6294.046875,0.347142,0.570385,0.082473,0.608609,0.171656
106,0.0,0.0,0.000000,58616.812500,0.000000,551.687256,62774.593750,0.0,64967.972656,64969.984375,...,296.736816,ZONE_107,107,2193.378906,5801.484375,0.008491,0.902214,0.089295,0.009412,0.000000
105,0.0,0.0,0.000000,11570.684570,1300.638306,58073.324219,52544.261719,0.0,74767.718750,74795.265625,...,114.413948,ZONE_106,106,20922.820312,5151.257812,0.776430,0.154698,0.068871,5.019005,1.739572
104,0.0,0.0,0.000000,82807.398438,0.000000,0.000000,111661.320312,0.0,130572.945312,130572.937500,...,10587.045898,ZONE_105,105,18911.625000,47765.539062,0.000000,0.634185,0.365815,0.000000,0.000000
103,0.0,0.0,0.000000,141717.187500,0.000000,0.000000,152252.234375,0.0,171114.390625,171114.421875,...,1003.809753,ZONE_104,104,18862.156250,29397.234375,0.000000,0.828201,0.171799,0.000000,0.000000
102,0.0,0.0,0.000000,100315.265625,0.000000,0.000000,102062.921875,0.0,141954.250000,141954.281250,...,764.253479,ZONE_103,103,39891.328125,41639.015625,0.000000,0.706673,0.293327,0.000000,0.000000
101,0.0,0.0,0.000000,50988.617188,0.000000,0.000000,51651.175781,0.0,57288.734375,57288.769531,...,285.031372,ZONE_102,102,5637.558594,6300.152344,0.000000,0.890028,0.109972,0.000000,0.000000
100,0.0,0.0,0.000000,63617.945312,7092.415039,142691.078125,197397.671875,0.0,232416.656250,232416.656250,...,852.450378,ZONE_101,101,27926.562500,26107.625000,0.613945,0.273724,0.112331,2.242938,3.051595
99,0.0,0.0,0.000000,245910.421875,0.000000,0.000000,261024.953125,0.0,280127.781250,280127.843750,...,3807.952881,ZONE_100,100,19102.828125,34217.421875,0.000000,0.877851,0.122149,0.000000,0.000000
